In [29]:
import numpy as np
import pandas as pd

In [30]:
from tensorflow.config.experimental import list_physical_devices, set_memory_growth
physical_devices = list_physical_devices('GPU')
set_memory_growth(physical_devices[0], True)

In [31]:
import tensorflow.keras.backend as K
K.clear_session()

In [32]:
train_data_v2 = np.load('../../data/training_data_v4.npy', allow_pickle=True)

In [33]:
df = pd.DataFrame(train_data_v2)
print(df.head())

                                                   0          1
0  [[11, 25, 22, 28, 29, 28, 25, 30, 30, 29, 22, ...  [0, 1, 0]
1  [[96, 74, 101, 164, 171, 171, 170, 169, 169, 1...  [1, 0, 0]
2  [[187, 186, 185, 185, 183, 183, 182, 181, 181,...  [0, 0, 1]
3  [[251, 251, 250, 250, 249, 249, 248, 248, 247,...  [1, 0, 0]
4  [[60, 113, 74, 48, 62, 52, 54, 42, 52, 101, 16...  [1, 0, 0]


In [34]:
# from keras_preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(rescale=1./255.,
#                              validation_split=0.2,
#                              rotation_range=10, # rotation
#                              width_shift_range=0.2, # horizontal shift
#                              height_shift_range=0.2, # vertical shift
#                              zoom_range=0.2, # zoom
#                              horizontal_flip=True, # horizontal flip
#                              brightness_range=[0.2,1.2]) # brightness)

In [35]:
# train_generator  = datagen.flow_from_dataframe(
#     dataframe=train_data_v2[1],
#     directory=save_dir,
#     x_col="path",
#     y_col="binary_score",
#     target_size=(80, 60),
#     color_mode="rgb",
#     batch_size=64,
#     save_to_dir='png_images_resized/',
#     save_format="png",
#     class_mode="binary",
#     subset='training',
#     shuffle = True
# )


# validation_generator  = datagen.flow_from_dataframe(
#     dataframe=train_clean,
#     directory=save_dir,
#     x_col="path",
#     y_col="binary_score",
#     target_size=(258, 258),
#     color_mode="rgb",
#     batch_size=64, #deberia ser mas alto que el del modelo (sugerencia)
#     save_to_dir='png_images_resized/',
#     save_format="png",
#     class_mode="binary",
#     subset='validation',
#     shuffle = True
# )

In [36]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')

    return model

In [37]:
WIDTH = 80
HEIGHT = 60
LR = 1e-3
EPOCHS = 12
MODEL_NAME = 'pygta5-car-{}-{}-{}-epochs.model'.format(LR, 'alexnetv2',EPOCHS)

model = alexnet(WIDTH, HEIGHT, LR)

In [41]:
train_data = np.load('../../data/training_data_v4.npy', allow_pickle=True)

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
test_y = [i[1] for i in test]


<ipython-input-41-e87fc2838f6e>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)


ValueError: cannot reshape array of size 50251 into shape (80,60,1)

In [23]:
# model.fit({'input': X}, {'targets': Y}, n_epoch=EPOCHS, validation_set=({'input': test_x}, {'targets': test_y}), 
#     snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 4667  | total loss: 0.22426 | time: 383.675s
| Momentum | epoch: 012 | loss: 0.22426 - acc: 0.9352 -- iter: 24832/24877
Training Step: 4668  | total loss: 0.21050 | time: 385.600s
| Momentum | epoch: 012 | loss: 0.21050 - acc: 0.9385 | val_loss: 0.29430 - val_acc: 0.9140 -- iter: 24877/24877
--


In [24]:
model.save(MODEL_NAME)

In [12]:
train_data_v2 = np.load('../../data/training_data_v2.npy', allow_pickle=True)

train_data_v3 = np.load('../../data/training_data_v3.npy', allow_pickle=True)

In [28]:
df_v2 = pd.DataFrame(train_data_v2)
print(df_v2.head())
df_v3 = pd.DataFrame(train_data_v3)
print(df_v3.head())

df_v2[0].shape
df_v3[0].shape

                                                   0          1
0  [[161, 160, 159, 159, 159, 158, 157, 157, 157,...  [1, 0, 0]
1  [[252, 252, 251, 251, 251, 250, 250, 249, 248,...  [0, 0, 1]
2  [[178, 177, 177, 177, 176, 176, 175, 174, 174,...  [0, 0, 1]
3  [[177, 176, 176, 175, 175, 175, 174, 174, 173,...  [0, 0, 1]
4  [[177, 177, 176, 176, 175, 175, 175, 174, 174,...  [0, 0, 1]
                                                   0          1
0  [[64, 169, 168, 167, 167, 167, 166, 166, 165, ...  [0, 0, 1]
1  [[204, 204, 202, 200, 198, 197, 196, 194, 192,...  [0, 0, 1]
2  [[175, 175, 174, 173, 173, 172, 171, 170, 169,...  [0, 0, 1]
3  [[233, 233, 232, 232, 232, 231, 230, 230, 230,...  [1, 0, 0]
4  [[238, 237, 236, 234, 234, 232, 231, 229, 227,...  [1, 0, 0]


(49506,)